In [9]:
%reset
import rasterio
from rasterio.features import shapes
import geopandas as gp
from shapely.geometry import Point

outpoly = "D:/cbr/VP2/VP2_SoilMapping/polygone.shp"
naip_fn = 'D:/cbr/VP2/VP2_SoilMapping/Processed/classified_Wetzwil_mitvierKlassen.tif'

mask = None
with rasterio.Env():
    with rasterio.open(naip_fn) as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))


geoms = list(results)
 # first feature
#print(geoms[0])

gpd_polygonized_raster  = gp.GeoDataFrame.from_features(geoms)
print(gpd_polygonized_raster)

#gpd_polygonized_raster['geometry'] = gpd_polygonized_raster.apply(lambda x: Point((float(x.lon), float(x.lat))), axis=1)
gpd_polygonized_raster.to_file(outpoly, driver='ESRI Shapefile')

                                              geometry  raster_val
0    POLYGON ((2652571.950 1230074.347, 2652571.950...         2.0
1    POLYGON ((2652647.174 1230074.347, 2652647.174...         3.0
2    POLYGON ((2652655.308 1230074.392, 2652656.162...         4.0
3    POLYGON ((2652535.821 1230074.347, 2652535.821...         2.0
4    POLYGON ((2652580.982 1230071.337, 2652580.982...         1.0
..                                                 ...         ...
458  POLYGON ((2652545.482 1229739.746, 2652545.482...         2.0
459  POLYGON ((2652548.403 1229739.612, 2652548.403...         2.0
460  POLYGON ((2652343.131 1230003.257, 2652343.131...     -9999.0
461  POLYGON ((2652466.618 1230116.498, 2652470.302...         2.0
462  POLYGON ((2652820.856 1229898.823, 2652820.946...     -9999.0

[463 rows x 2 columns]


AttributeError: 'Series' object has no attribute 'lon'

In [ ]:
from osgeo import gdal, ogr, os

outpoly = "D:/cbr/VP2/VP2_SoilMapping/polygone.shp"
naip_fn = 'D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/prediction2/Prediction.tif'

sourceRaster = gdal.Open(naip_fn)
band = sourceRaster.GetRasterBand(1)
bandArray = band.ReadAsArray()
driver = ogr.GetDriverByName("ESRI Shapefile")
if os.path.exists(outpoly):
    driver.DeleteDataSource(outpoly)
outDatasource = driver.CreateDataSource(outpoly)
outLayer = outDatasource.CreateLayer("polygonized", srs=None)
newField = ogr.FieldDefn('MYFLD')
outLayer.CreateField(newField)
gdal.Polygonize(band, None, outLayer, 0, [], callback=None )
outDatasource.Destroy()
sourceRaster = None


AttributeError: 'NoneType' object has no attribute 'CreateLayer'